In [3]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [4]:
import os

def langsmith(project_name=None, set_enable=True):

    if set_enable:
        result = os.environ.get("LANGCHAIN_API_KEY")
        if result is None or result.strip() == "":
            print(
                "LangChain API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954"
            )
            return
        os.environ["LANGCHAIN_ENDPOINT"] = (
            "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        )
        os.environ["LANGCHAIN_TRACING_V2"] = "true"  # true: 활성화
        os.environ["LANGCHAIN_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name}")
    else:
        os.environ["LANGCHAIN_TRACING_V2"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")


langsmith("STUDY-01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
STUDY-01-Basic


## Azure OpenAI
Azure OpenAI란 Azure 클라우드 플랫폼에서 제공하는 OpenAI 서비스를 말합니다.

객체를 생성할 때 다음 옵션을 값을 지정 할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다:

`temperature`

- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`

- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`model_name`: 적용 가능한 모델 리스트
- `gpt-4o`
- `gpt-4o-mini`


In [18]:
from langchain_openai import AzureChatOpenAI

# 객체 생성
llm = AzureChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,  # 창의성 (0.0 ~ 2.0)
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울입니다.' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 16, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_d54531d9eb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'profanity': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'profanity': {'filtered': False, 'detected': False}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'sev

### 답변의 형식(AI Message)

In [19]:
# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
response = llm.invoke(question)

In [20]:
response

AIMessage(content='대한민국의 수도는 서울입니다.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 16, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_d54531d9eb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'profanity': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'profanity': {'filtered': False, 'detected': False}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False,

In [21]:
response.content

'대한민국의 수도는 서울입니다.'

In [22]:
response.response_metadata

{'token_usage': {'completion_tokens': 8,
  'prompt_tokens': 16,
  'total_tokens': 24,
  'completion_tokens_details': None,
  'prompt_tokens_details': None},
 'model_name': 'gpt-4o-mini',
 'system_fingerprint': 'fp_d54531d9eb',
 'prompt_filter_results': [{'prompt_index': 0,
   'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'},
    'jailbreak': {'filtered': False, 'detected': False},
    'profanity': {'filtered': False, 'detected': False},
    'self_harm': {'filtered': False, 'severity': 'safe'},
    'sexual': {'filtered': False, 'severity': 'safe'},
    'violence': {'filtered': False, 'severity': 'safe'}}}],
 'finish_reason': 'stop',
 'logprobs': None,
 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'},
  'profanity': {'filtered': False, 'detected': False},
  'protected_material_code': {'filtered': False, 'detected': False},
  'protected_material_text': {'filtered': False, 'detected': False},
  'self_harm': {'filtered': False, 'severity'

### 스트리밍 출력

스트리밍 옵션은 AI 답변을 실시간으로 받을 수 있습니다.

In [23]:
# 스트림 방식으로 질의
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")

In [24]:
# 스트리밍 방식으로 각 토큰을 출력합니다. (실시간 출력)

for token in answer:
    print(token.content, end="", flush=True)

대한민국에는 아름다운 관광지가 많이 있습니다. 아래는 추천하는 10곳과 그 주소입니다.

1. **경복궁**
   - 주소: 서울특별시 종로구 사직로 161

2. **제주도**
   - 주소: 제주특별자치도 제주시

3. **부산 해운대 해수욕장**
   - 주소: 부산광역시 해운대구 해운대해변로 264

4. **경주 불국사**
   - 주소: 경상북도 경주시 불국로 385

5. **남이섬**
   - 주소: 강원도 춘천시 남이섬길 1

6. **전주 한옥마을**
   - 주소: 전라북도 전주시 완산구 기린대로 99

7. **설악산 국립공원**
   - 주소: 강원도 속초시 설악산로 173

8. **안동 하회마을**
   - 주소: 경상북도 안동시 풍천면 하회리

9. **서울 남산타워 (N서울타워)**
   - 주소: 서울특별시 용산구 남산공원길 105

10. **광주 무등산**
    - 주소: 광주광역시 동구 무등산로 100

이 외에도 많은 아름다운 관광지가 있으니, 방문 계획에 참고하시기 바랍니다!

In [26]:
from langchain_teddynote.messages import stream_response

# 스트림 방식으로 질의
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")
stream_response(answer)

대한민국에는 아름다운 관광지가 많이 있습니다. 아래는 추천하는 10곳과 그 주소입니다.

1. **경복궁**
   - 주소: 서울특별시 종로구 사직로 161

2. **제주도**
   - 주소: 제주특별자치도 제주시

3. **부산 해운대 해수욕장**
   - 주소: 부산광역시 해운대구 해운대해변로 140

4. **경주 불국사**
   - 주소: 경상북도 경주시 불국로 385

5. **남이섬**
   - 주소: 강원도 춘천시 남이섬길 1

6. **전주 한옥마을**
   - 주소: 전라북도 전주시 완산구 기린대로 99

7. **설악산 국립공원**
   - 주소: 강원도 속초시 설악산로 173

8. **안동 하회마을**
   - 주소: 경상북도 안동시 풍천면 하회리

9. **서울 남산타워 (N서울타워)**
   - 주소: 서울특별시 용산구 남산공원길 105

10. **광주 무등산**
    - 주소: 광주광역시 동구 무등로 100

각 관광지는 독특한 매력을 가지고 있으며, 방문 시 다양한 문화와 자연을 경험할 수 있습니다.